In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib_venn import venn2, venn2_circles, venn3
import sys
sys.path.append("/home/danya-2003/pyvenn")
import venn

In [2]:
df1 = pd.read_table('/home/danya-2003/Downloads/ctd/ctd_switched_promoters/Kulakovsky/CTD2587H24_ASO_G0267577_01.tsv', sep='\t')
df2 = pd.read_table('/home/danya-2003/Downloads/ctd/ctd_switched_promoters/Kulakovsky/CTD2587H24_ASO_G0267577_03.tsv', sep='\t')
df1 = df1.set_index(keys = 'prmtrID', drop=False)
df2 = df2.set_index(keys = 'prmtrID', drop=False)

In [5]:
# table with indexes (prmtrID) and columns geneID and prmtrID
def get_df_for_dict(df):
    df_for_dict = df[["geneID", "prmtrID"]]
    return df_for_dict

# dictionary with keys - genes, and values - lists of their promoters
def get_df_dict(df_for_dict):
    df_dict = {}
    for index, row in df_for_dict.iterrows():
        gene_id = df_for_dict.loc[index].values[0]
        if gene_id not in df_dict:
            df_dict[gene_id] = list()
        df_dict[gene_id].append(index)
    return df_dict

#table with columns chr, start, end and strand to find distances
def get_df_coor(df):
    df_coor = df
    df_coor[['chr','start', 'end', 'strand']] = df_coor['prmtrID'].str.split('_',expand=True)
    df_coor = df_coor[['geneID', 'prmtrID', 'prmtrCoef', 'chr','start', 'end', 'strand']]
    return df_coor

#table with promoters of one gene with distance from 150 to 2000

def get_distanted (df_dict, df_coor):
    
    distanced = pd.DataFrame(columns=['gene', 'prmtr'])
    for key in df_dict:
        promoters=df_dict.get(key)
        dim = len(promoters)
        dist = pd.DataFrame(columns=promoters,
                      index=promoters)
        for col in dist:
            a_start = int(df_coor.loc[col].values[4])
            a_end = int(df_coor.loc[col].values[5])
            for row in dist:
                b_start = int(df_coor.loc[row].values[4])
                b_end = int(df_coor.loc[row].values[5])
            
                if (a_start < b_start) & (a_end < b_start):
                    dist.at[col, row] = b_start - a_end
                elif (b_start < a_start) & (b_end < a_start):
                    dist.at[col, row] = a_start - b_end
                else:
                    dist.at[col, row] = 0
        
        dist[dist < 150] = 0
        dist[dist > 2000] = 0
        
        v = dist.sum(axis=0)
        
        for row in v.index:                
            if v[row] > 0:
                #new_row = {'gene':key, 'prmtr':row}
                #distanced=distanced.append(new_row, ignore_index=True)
                new_row = [key, row]
                distanced.loc[len(distanced)] = new_row


    return distanced

In [6]:
# get filtered tables with prmtrs only with distance >150 and <2000 n
df_for_dict_1 = get_df_for_dict(df1)
dict_df1 = get_df_dict(df_for_dict_1)
coor_df1 = get_df_coor(df1)
distanted_df1 = get_distanted(dict_df1,coor_df1)
df1_dist = df1[df1.index.isin(distanted_df1.prmtr.tolist())]

df_for_dict_2 = get_df_for_dict(df2)
dict_df2 = get_df_dict(df_for_dict_2)
coor_df2 = get_df_coor(df2)
distanted_df2 = get_distanted(dict_df2,coor_df2)
df2_dist = df2[df2.index.isin(distanted_df2.prmtr.tolist())]

In [11]:
#table with switched promoters; FDR < 0.05; prmtrPV < 0.05
def get_switched(df_dist):
    switched = df_dist[(df_dist["geneFDR"] < 0.05) & (df_dist["prmtrPv"] < 0.05)]
    switched = switched[["geneID", "prmtrID", "prmtrCoef"]]
    return switched

#table with unswitched promoters; FDR < 0.05; prmtrPV >= 0.05
def get_unswitched(df_dist):
    unswitched = df_dist[(df_dist["geneFDR"] < 0.05) | (df_dist["prmtrPv"] > 0.05)]
    unswitched = unswitched[["geneID", "prmtrID", "prmtrCoef"]]
    return unswitched

In [12]:
# get switched and unswitched prmtrs
switched_df1 = get_switched(df1_dist)
switched_df2 = get_switched(df2_dist)
unswitched_df1 = get_unswitched(df1_dist)
unswitched_df2 = get_unswitched(df2_dist)

# switch on and switch off

switch_plus_01 = switched_df1[switched_df1['prmtrCoef'] > 0 ]
switch_minus_01 = switched_df1[switched_df1['prmtrCoef'] < 0 ]
switch_plus_03 = switched_df2[switched_df2['prmtrCoef'] > 0 ]
switch_minus_03 = switched_df2[switched_df2['prmtrCoef'] < 0 ]

# get COMMON switched and unswitched prmtrs

switched = switched_df1[switched_df1.prmtrID.isin(switched_df2.prmtrID.tolist())]
unswitched = unswitched_df1[unswitched_df1.prmtrID.isin(unswitched_df2.prmtrID.tolist())]

In [ ]:

def cpm_genes(df_cpm, df, df_result):
    df_cpm_all = df_cpm[['ID', 'mean']].set_index(keys = 'ID', drop=False)
    df_cpm_all = df_cpm_all[df_cpm_all.index.isin(df.prmtrID.tolist())]
    df_cpm_all[['gene']] = ''
    for index, row in df_cpm_all.iterrows():
        df_cpm_all['gene'][index] = df.loc[index].values[0]
    return df_cpm_all

    cpm_dict = {}
    for index, row in df_cpm_all.iterrows():
        gene_id=df_cpm_all.loc[index][2]

        if gene_id not in cpm_dict:
            cpm_dict[gene_id] = list()
        cpm_dict[gene_id].append(df_cpm_all.loc[index][1])

    for key in cpm_dict:
        cpm_dict[key] = sum(cpm_dict[key])

    df_cpm_all[['sum']] = ''

    for index, row in df_cpm_all.iterrows():
        gene = df_cpm_all['gene'][index]
        summ = cpm_dict[gene]
        df_cpm_all['sum'][index] = summ

    df_cpm_all['cntrb'] = df_cpm_all['mean'] / df_cpm_all['sum']

    df_cpm_filtered = df_cpm_all[(df_cpm_all['cntrb'] > 0.3)]
    
    df_cpm_filtered = df_cpm_filtered[df_cpm_filtered.index.isin(df_result.prmtrID.tolist())]
    
    return df_cpm_filtered
#calculate contribution of each promoter to gene expression and filter by contribution > 0.3

In [ ]:
df1_cpm = pd.read_table('/home/danya-2003/Downloads/ctd/ctd_switched_promoters/Kulakovsky/CTD-2587H24.5_ASO_G0267577_01_CPM.tsv', sep='\t')
df2_cpm = pd.read_table('/home/danya-2003/Downloads/ctd/ctd_switched_promoters/Kulakovsky/CTD-2587H24.5_ASO_G0267577_03_CPM.tsv', sep='\t')

In [ ]:
df2_cpm

In [ ]:
df1_cpm = pd.read_table('/home/danya-2003/Downloads/ctd/ctd_switched_promoters/Kulakovsky/CTD-2587H24.5_ASO_G0267577_01_CPM.tsv', sep='\t')
df2_cpm = pd.read_table('/home/danya-2003/Downloads/ctd/ctd_switched_promoters/Kulakovsky/CTD-2587H24.5_ASO_G0267577_03_CPM.tsv', sep='\t')
ID = df1_cpm['ID'].tolist()
col_1 = df1_cpm['CNhi10491_ACC'].tolist()
col_2 = df1_cpm['CNhi10492_GCT'].tolist()
col_3 = df2_cpm['CNhi10487_ACC'].tolist() 
col_4 = df2_cpm['CNhi10495_ACG'].tolist()
df0_cpm = pd.DataFrame({'ID': ID, 'col1': col_1, 'col2': col_2, 'col3': col_3, 'col4': col_4})
df0_cpm['mean'] = (df0_cpm['col1'] + df0_cpm['col2'] + df0_cpm['col3'] + df0_cpm['col4']) / 4
df1_cpm['mean'] = (df1_cpm['CNhi10491_ACC'] + df1_cpm['CNhi10492_GCT']) / 2
df2_cpm['mean'] = (df2_cpm['CNhi10487_ACC'] + df2_cpm['CNhi10495_ACG']) / 2

In [ ]:
# get switched and unswitched prmtrs
switched_df1 = get_switched(df1_dist)
switched_df2 = get_switched(df2_dist)
unswitched_df1 = get_unswitched(df1_dist)
unswitched_df2 = get_unswitched(df2_dist)

# switch on and switch off

switch_plus_01 = switched_df1[switched_df1['prmtrCoef'] > 0 ]
switch_minus_01 = switched_df1[switched_df1['prmtrCoef'] < 0 ]
switch_plus_03 = switched_df2[switched_df2['prmtrCoef'] > 0 ]
switch_minus_03 = switched_df2[switched_df2['prmtrCoef'] < 0 ]

# get COMMON switched and unswitched prmtrs

switched = switched_df1[switched_df1.prmtrID.isin(switched_df2.prmtrID.tolist())]
unswitched = unswitched_df1[unswitched_df1.prmtrID.isin(unswitched_df2.prmtrID.tolist())]

#df=df1_dist[df1_dist.index.isin(df2.prmtrID.tolist())]
#df_for_dict = get_df_for_dict(df)
#dict_df = get_df_dict(df_for_dict)
#coor_df = get_df_coor(df)
#distanted_df = get_distanted(dict_df,coor_df)
#df_dist = df[df.index.isin(distanted_df.prmtr.tolist())]
#df_switched = get_df_unswitched(df)
#df_unswitched = get_unswitched(df)

df0_cpm_filtered = cpm(df0_cpm, df, df0_result)
#df1_cpm_filtered = cpm(df1_cpm, df1, df1_result)
#df2_cpm_filtered = cpm(df2_cpm, df2, df2_result)
df0_cpm_result = df_coor[df_coor.index.isin(df0_cpm_filtered.ID.tolist())]
#df1_cpm_result = df1_coor[df1_coor.index.isin(df1_cpm_filtered.ID.tolist())]
#df2_cpm_result = df2_coor[df2_coor.index.isin(df2_cpm_filtered.ID.tolist())]

In [ ]:
# save files

ctd_switch = switched['prmtrID'].tolist()
with open("ctd_switch.txt", 'w') as f:
  for i in range (len(ctd_switch)):
    f.write(ctd_switch[i]+ '\n')

ctd_unswitch = unswitched['prmtrID'].tolist()
with open("ctd_unswitch.txt", 'w') as f:
  for i in range (len(ctd_unswitch)):
    f.write(ctd_unswitch[i]+ '\n')
    
ctd_switch_01 = switched_df1['prmtrID'].tolist()
with open("ctd_switch_01.txt", 'w') as f:
  for i in range (len(ctd_switch_01)):
    f.write(ctd_switch_01[i]+ '\n')
    
ctd_switch_03 = switched_df2['prmtrID'].tolist()
with open("ctd_switch_03.txt", 'w') as f:
  for i in range (len(ctd_switch_03)):
    f.write(ctd_switch_03[i]+ '\n')
    
ctd_unswitch_01 = unswitched_df1['prmtrID'].tolist()
with open("ctd_unswitch_01.txt", 'w') as f:
  for i in range (len(ctd_unswitch_01)):
    f.write(ctd_unswitch_01[i]+ '\n')
    
ctd_unswitch_03 = unswitched_df2['prmtrID'].tolist()
with open("ctd_unswitch_03.txt", 'w') as f:
  for i in range (len(ctd_unswitch_03)):
    f.write(ctd_unswitch_03[i]+ '\n')


ctd_switch_plus_01 = switch_plus_01['prmtrID'].tolist()
with open("ctd_switch_plus_01.txt", 'w') as f:
  for i in range (len(ctd_switch_plus_01)):
    f.write(ctd_switch_plus_01[i]+ '\n')

ctd_switch_minus_01 = switch_minus_01['prmtrID'].tolist()
with open("ctd_switch_minus_01.txt", 'w') as f:
  for i in range (len(ctd_switch_minus_01)):
    f.write(ctd_switch_minus_01[i]+ '\n')    
    
ctd_switch_plus_03 = switch_plus_03['prmtrID'].tolist()
with open("ctd_switch_plus_03.txt", 'w') as f:
  for i in range (len(ctd_switch_plus_03)):
    f.write(ctd_switch_plus_03[i]+ '\n')

ctd_switch_minus_03 = switch_minus_03['prmtrID'].tolist()
with open("ctd_switch_minus_03.txt", 'w') as f:
  for i in range (len(ctd_switch_minus_03)):
    f.write(ctd_switch_minus_03[i]+ '\n')

In [ ]:
ctd_cpm=df0_cpm_result['prmtrID'].tolist()
with open("ctd_cpm.txt", 'w') as f:
  for i in range (len(ctd_cpm)):
    f.write(ctd_cpm[i]+ '\n')    
    
ctd_cpm_minus=df0_cpm_minus['prmtrID'].tolist()
with open("ctd_cpm_minus.txt", 'w') as f:
  for i in range (len(ctd_cpm_minus)):
    f.write(ctd_cpm_minus[i]+ '\n')
    
ctd_cpm_plus=df0_cpm_plus['prmtrID'].tolist()
with open("ctd_cpm_plus.txt", 'w') as f:
  for i in range (len(ctd_cpm_plus)):
    f.write(ctd_cpm_plus[i]+ '\n')

In [ ]:
ctd_cpm_01=df1_cpm_result['prmtrID'].tolist()
with open("ctd_cpm_01.txt", 'w') as f:
  for i in range (len(ctd_cpm_01)):
    f.write(ctd_cpm_01[i]+ '\n')

ctd_cpm_plus_01=df1_cpm_plus['prmtrID'].tolist()
with open("ctd_cpm_plus_01.txt", 'w') as f:
  for i in range (len(ctd_cpm_plus_01)):
    f.write(ctd_cpm_plus_01[i]+ '\n')

ctd_cpm_minus_01=df1_cpm_minus['prmtrID'].tolist()
with open("ctd_cpm_minus_01.txt", 'w') as f:
  for i in range (len(ctd_cpm_minus_01)):
    f.write(ctd_cpm_minus_01[i]+ '\n')  
    
ctd_cpm_03=df2_cpm_result['prmtrID'].tolist()
with open("ctd_cpm_03.txt", 'w') as f:
  for i in range (len(ctd_cpm_03)):
    f.write(ctd_cpm_03[i]+ '\n') 
    
ctd_cpm_plus_03=df2_cpm_plus['prmtrID'].tolist()
with open("ctd_cpm_plus_03.txt", 'w') as f:
  for i in range (len(ctd_cpm_plus_03)):
    f.write(ctd_cpm_plus_03[i]+ '\n')

ctd_cpm_minus_03=df2_cpm_minus['prmtrID'].tolist()
with open("ctd_cpm_minus_03.txt", 'w') as f:
  for i in range (len(ctd_cpm_minus_03)):
    f.write(ctd_cpm_minus_03[i]+ '\n')


In [ ]:
ctd_ensg_cpm_plus_01=df1_cpm_plus['geneID'].tolist()
with open("ctd_ensg_cpm_plus_01.txt", 'w') as f:
  for i in range (len(ctd_ensg_cpm_plus_01)):
    f.write(ctd_ensg_cpm_plus_01[i]+ '\n')

ctd_ensg_cpm_minus_01=df1_cpm_minus['geneID'].tolist()
with open("ctd_ensg_cpm_minus_01.txt", 'w') as f:
  for i in range (len(ctd_ensg_cpm_minus_01)):
    f.write(ctd_ensg_cpm_minus_01[i]+ '\n')    
    
ctd_ensg_cpm_plus_03=df2_cpm_plus['geneID'].tolist()
with open("ctd_ensg_cpm_plus_03.txt", 'w') as f:
  for i in range (len(ctd_ensg_cpm_plus_03)):
    f.write(ctd_ensg_cpm_plus_03[i]+ '\n')

ctd_ensg_cpm_minus_03=df2_cpm_minus['geneID'].tolist()
with open("ctd_ensg_cpm_minus_03.txt", 'w') as f:
  for i in range (len(ctd_ensg_cpm_minus_03)):
    f.write(ctd_ensg_cpm_minus_03[i]+ '\n')
    
ctd_ensg_cpm_minus=df0_cpm_minus['geneID'].tolist()
with open("ctd_ensg_cpm_minus.txt", 'w') as f:
  for i in range (len(ctd_ensg_cpm_minus)):
    f.write(ctd_ensg_cpm_minus[i]+ '\n')
    
ctd_ensg_cpm_plus=df0_cpm_plus['geneID'].tolist()
with open("ctd_ensg_cpm_plus.txt", 'w') as f:
  for i in range (len(ctd_ensg_cpm_plus)):
    f.write(ctd_ensg_cpm_plus[i]+ '\n')

In [ ]:
#find KDM4A promoters
for i, elem in enumerate(df1['geneID']):
        if 'ENSG00000066135' in elem:
            print(df1.iloc[i])

In [ ]:
df1_minus.loc['ENSG00000066135']
#p2

In [ ]:
df1_plus.loc['ENSG00000066135']
#p4

In [ ]:
df2_minus.loc['ENSG00000066135']

In [ ]:
df2_plus.loc['ENSG00000066135']

In [ ]:
print(len(df1_result_minus['prmtrID'].tolist()))
print(len(df2_result_minus['prmtrID'].tolist()))
print(len(df0_result_minus['prmtrID'].tolist()))
print(len(df1_cpm_minus['prmtrID'].tolist()))
print(len(df2_cpm_minus['prmtrID'].tolist()))
print(len(df0_cpm_minus['prmtrID'].tolist()))

print(len(df1_result_plus['prmtrID'].tolist()))
print(len(df2_result_plus['prmtrID'].tolist()))
print(len(df0_result_plus['prmtrID'].tolist()))
print(len(df1_cpm_plus['prmtrID'].tolist()))
print(len(df2_cpm_plus['prmtrID'].tolist()))
print(len(df0_cpm_plus['prmtrID'].tolist()))

In [ ]:
df1_gene = set(df1['geneID'])
print('df1_gene', len(df1_gene))

df2_gene = set(df2['geneID'])
print('df2_gene', len(df2_gene))

df1_gene_filtered = set(df1_filtered['geneID'])
print('df1_gene_filtered', len(df1_gene_filtered))

df2_gene_filtered = set(df2_filtered['geneID'])
print('df2_gene_filtered', len(df2_gene_filtered))

common_gene = common_elements(df1_gene, df2_gene)
print('common_gene', len(common_gene))

common_gene_filtered = common_elements(df1_gene_filtered, df2_gene_filtered)
print('common_gene_filtered', len(common_gene_filtered))

print('saved_common_gene', len(common_elements(common_gene, common_gene_filtered)))


common_prmtr = common_elements(df1['prmtrID'].tolist(), df2['prmtrID'].tolist())
print('common_prmtr', len(common_prmtr))

common_prmtr_filtered = common_elements(df1_filtered['prmtrID'].tolist(), df2_filtered['prmtrID'].tolist())

print('common_prmtr_filtered', len(common_prmtr_filtered))

print('saved_common_prmtr', len(common_elements(common_prmtr, common_prmtr_filtered)))

In [15]:
df1_distances = get_distances(df1_dict, df1_coor)
df1_distances_cleaned = [x for x in df1_distances if str(x) != 'nan']
zero1=0
for i in df1_distances_cleaned:
    if i == 0:
        zero1+=1
df1_distances_cleaned = [i for i in df1_distances_cleaned if i != 0]
df1_distances_cleaned.sort()
distances_1 = pd.DataFrame({'promoters': df1_distances_cleaned})

df2_distances = get_distances(df2_dict, df2_coor)
df2_distances_cleaned = [x for x in df2_distances if str(x) != 'nan']
zero2=0
for i in df2_distances_cleaned:
    if i == 0:
        zero2+=1
df2_distances_cleaned = [i for i in df2_distances_cleaned if i != 0]
distances_2 = pd.DataFrame({'promoters': df2_distances_cleaned})

print('zero_distances_1', zero1)
print('zero_distances_2', zero2)

NameError: name 'get_distances' is not defined

In [ ]:
sns.histplot(data=distances_1)

In [ ]:
sns.histplot(data=distances_2)

In [ ]:
large1=0
for i in df1_distances_cleaned:
    if i > 100000:
        large1+=1
df1_distances_cleaned = [i for i in df1_distances_cleaned if i < 100000]
distances_1 = pd.DataFrame({'promoters': df1_distances_cleaned})

large2=0
for i in df2_distances_cleaned:
    if i > 100000:
        large2+=1
df2_distances_cleaned = [i for i in df2_distances_cleaned if i < 100000]
distances_2 = pd.DataFrame({'promoters': df2_distances_cleaned})

print('large_distances_1', large1)
print('large_distances_2', large2)

In [ ]:
sns.histplot(data=distances_1)

In [ ]:
sns.histplot(data=distances_2)

In [ ]:
large1=0
for i in df1_distances_cleaned:
    if i > 2000:
        large1+=1
df1_distances_cleaned = [i for i in df1_distances_cleaned if i < 2000]
distances_1 = pd.DataFrame({'promoters': df1_distances_cleaned})

large2=0
for i in df2_distances_cleaned:
    if i > 2000:
        large2+=1
df2_distances_cleaned = [i for i in df2_distances_cleaned if i < 2000]
distances_2 = pd.DataFrame({'promoters': df2_distances_cleaned})

print('large_distances_1', large1)
print('large_distances_2', large2)

In [ ]:
sns.histplot(data=distances_1)

In [ ]:
sns.histplot(data=distances_2)

In [ ]:
df1_item_num = item_num(df1_dict)
df2_item_num = item_num(df2_dict)

In [ ]:
sns.histplot(data=df1_item_num)

In [ ]:
sns.histplot(data=df1_item_num)

In [ ]:
for_zenbu = get_distanted(df2_dict, df2_coor, 150, 1500)
for_zenbu

In [ ]:
            if (a_start > b_start) & (a_end < b_end):
                dist.at['col', 'row'] = 0
            if (b_start > a_start) & (b_end < a_end):
                dist.at['col', 'row'] = 0
            if (a_start < b_start) & (a_end < b_end) & (a_end > b_start):
                dist.at['col', 'row'] = 0
            if (b_start < a_start) & (b_end < a_end) & (b_end > a_start):
                dist.at['col', 'row'] = 0
            if (a_start == b_start):
                dist.at['col', 'row'] = 0
            if (a_end == b_end):
                dist.at['col', 'row'] = 0

In [ ]:
df2_unswitched = get_df_unswitched(df2)
df2_unswitched = df2_unswitched[['geneID','prmtrID','prmtrCoef']].set_index(keys = 'prmtrID', drop=False)

df2_unswitched_dict = get_df_dict(df2_unswitched)
prmtrs_1 = []
prmtrs_2 = []
prmtrs_3 = []
prmtrs_4 = []
prmtrs_5 = []
for key in df2_unswitched_dict:
    prmtrs_list = df2_unswitched_dict.get(key)
    prmtrs_list.sort()
    item = 0
    for i in prmtrs_list:
        item += 1
        if item == 1:
            prmtrs_1.append(i)
        if item == 2:
            prmtrs_2.append(i)
        if item == 3:
            prmtrs_3.append(i)
        if item == 4:
            prmtrs_4.append(i)
        if item >= 5:
            prmtrs_5.append(i)
prmtrs_1_expr = pd.DataFrame(data={'prmtr': prmtrs_1})
prmtrs_2_expr = pd.DataFrame(data={'prmtr': prmtrs_2})
prmtrs_3_expr = pd.DataFrame(data={'prmtr': prmtrs_3})
prmtrs_4_expr = pd.DataFrame(data={'prmtr': prmtrs_4})
prmtrs_5_expr = pd.DataFrame(data={'prmtr': prmtrs_5})

prmtrs_1_expr['prmtr_num']='1st_prmtr'
prmtrs_2_expr['prmtr_num']='2nd_prmtr'
prmtrs_3_expr['prmtr_num']='3rd_prmtr'
prmtrs_4_expr['prmtr_num']='4th_prmtr'
prmtrs_5_expr['prmtr_num']='5th-7th_prmtr'

prmtrs_expr = pd.concat([prmtrs_1_expr, prmtrs_2_expr, prmtrs_3_expr, 
                         prmtrs_4_expr, prmtrs_5_expr], axis = 0, ignore_index = True)
prmtrs_expr['prmtr_num']=prmtrs_expr['prmtr_num']
prmtrs_expr['geneID']=''
prmtrs_expr['coef']=''
prmtrs_expr['cntrb']=''
prmtrs_expr = prmtrs_expr.set_index(keys = 'prmtr', drop=False)

for index, row in prmtrs_expr.iterrows():
    prmtr_id = prmtrs_expr.loc[index][0]
    prmtrs_expr.loc[index][2] = df2_unswitched.loc[prmtr_id][0]
    prmtrs_expr.loc[index][3] = df2_unswitched.loc[prmtr_id][2]

df_cpm = df2_cpm[['ID', 'mean']].set_index(keys = 'ID', drop=False)
df_cpm = df_cpm[df_cpm.index.isin(prmtrs_expr.prmtr.tolist())]
df_cpm[['gene']] = ''
df_cpm[['sum']] = ''
for index, row in df_cpm.iterrows():
    df_cpm['gene'][index] = prmtrs_expr.loc[index][2]

cpm_dict = {}
for index, row in df_cpm.iterrows():
    gene_id=df_cpm.loc[index][2]
    if gene_id not in cpm_dict:
        cpm_dict[gene_id] = list()
    cpm_dict[gene_id].append(df_cpm.loc[index][1])
    
for key in cpm_dict:
    cpm_dict[key] = sum(cpm_dict[key])
    
for index, row in df_cpm.iterrows():
    gene = df_cpm['gene'][index]
    summ = cpm_dict[gene]
    df_cpm['sum'][index] = summ
    
df_cpm['cntrb'] = df_cpm['mean'] / df_cpm['sum']

for index, row in prmtrs_expr.iterrows():
    prmtr_id = prmtrs_expr.loc[index][0]
    prmtrs_expr.loc[index][4] = df_cpm.loc[prmtr_id][4]

prmtrs_expr['prmtr']=prmtrs_expr['prmtr'].astype('category')
prmtrs_expr['prmtr_num']=prmtrs_expr['prmtr_num'].astype('category')
prmtrs_expr['geneID']=prmtrs_expr['geneID'].astype('category')
prmtrs_expr['coef']=prmtrs_expr['coef'].astype(float)
prmtrs_expr['cntrb']=prmtrs_expr['cntrb'].astype(float)
prmtrs_expr_df2_uns=prmtrs_expr

In [ ]:
rcParams['figure.figsize'] = 12,12
rcParams["patch.force_edgecolor"] = True
ax = sns.violinplot(data = prmtrs_expr_df2_uns, x = 'prmtr_num', y = "coef", inner = 'quartile', palette = 'Set3', saturation = 1)
fig = ax.get_figure()
fig.patch.set_alpha(0)
fig.savefig("prmtrs_expr_df2_uns_coef.png", bbox_inches='tight', pad_inches =0)

In [ ]:
rcParams['figure.figsize'] = 12,12
rcParams["patch.force_edgecolor"] = True
ax = sns.violinplot(data = prmtrs_expr_df2_uns, x = 'prmtr_num', y = "cntrb", inner = 'quartile', palette = 'Set3', saturation = 1)
fig = ax.get_figure()
fig.patch.set_alpha(0)
fig.savefig("prmtrs_expr_df2_uns_cntrb.png", bbox_inches='tight', pad_inches =0)

In [ ]:
df1_unswitched = get_df_unswitched(df1)
df1_unswitched = df1_unswitched[['geneID','prmtrID','prmtrCoef']].set_index(keys = 'prmtrID', drop=False)

df1_unswitched_dict = get_df_dict(df1_unswitched)
prmtrs_1 = []
prmtrs_2 = []
prmtrs_3 = []
prmtrs_4 = []
prmtrs_5 = []
for key in df1_unswitched_dict:
    prmtrs_list = df1_unswitched_dict.get(key)
    prmtrs_list.sort()
    item = 0
    for i in prmtrs_list:
        item += 1
        if item == 1:
            prmtrs_1.append(i)
        if item == 2:
            prmtrs_2.append(i)
        if item == 3:
            prmtrs_3.append(i)
        if item == 4:
            prmtrs_4.append(i)
        if item >= 5:
            prmtrs_5.append(i)
prmtrs_1_expr = pd.DataFrame(data={'prmtr': prmtrs_1})
prmtrs_2_expr = pd.DataFrame(data={'prmtr': prmtrs_2})
prmtrs_3_expr = pd.DataFrame(data={'prmtr': prmtrs_3})
prmtrs_4_expr = pd.DataFrame(data={'prmtr': prmtrs_4})
prmtrs_5_expr = pd.DataFrame(data={'prmtr': prmtrs_5})

prmtrs_1_expr['prmtr_num']='1st_prmtr'
prmtrs_2_expr['prmtr_num']='2nd_prmtr'
prmtrs_3_expr['prmtr_num']='3rd_prmtr'
prmtrs_4_expr['prmtr_num']='4th_prmtr'
prmtrs_5_expr['prmtr_num']='5th-7th_prmtr'

prmtrs_expr = pd.concat([prmtrs_1_expr, prmtrs_2_expr, prmtrs_3_expr, 
                         prmtrs_4_expr, prmtrs_5_expr], axis = 0, ignore_index = True)
prmtrs_expr['prmtr_num']=prmtrs_expr['prmtr_num']
prmtrs_expr['geneID']=''
prmtrs_expr['coef']=''
prmtrs_expr['cntrb']=''
prmtrs_expr = prmtrs_expr.set_index(keys = 'prmtr', drop=False)

for index, row in prmtrs_expr.iterrows():
    prmtr_id = prmtrs_expr.loc[index][0]
    prmtrs_expr.loc[index][2] = df1_unswitched.loc[prmtr_id][0]
    prmtrs_expr.loc[index][3] = df1_unswitched.loc[prmtr_id][2]

df_cpm = df1_cpm[['ID', 'mean']].set_index(keys = 'ID', drop=False)
df_cpm = df_cpm[df_cpm.index.isin(prmtrs_expr.prmtr.tolist())]
df_cpm[['gene']] = ''
df_cpm[['sum']] = ''
for index, row in df_cpm.iterrows():
    df_cpm['gene'][index] = prmtrs_expr.loc[index][2]

cpm_dict = {}
for index, row in df_cpm.iterrows():
    gene_id=df_cpm.loc[index][2]
    if gene_id not in cpm_dict:
        cpm_dict[gene_id] = list()
    cpm_dict[gene_id].append(df_cpm.loc[index][1])
    
for key in cpm_dict:
    cpm_dict[key] = sum(cpm_dict[key])
    
for index, row in df_cpm.iterrows():
    gene = df_cpm['gene'][index]
    summ = cpm_dict[gene]
    df_cpm['sum'][index] = summ
    
df_cpm['cntrb'] = df_cpm['mean'] / df_cpm['sum']

for index, row in prmtrs_expr.iterrows():
    prmtr_id = prmtrs_expr.loc[index][0]
    prmtrs_expr.loc[index][4] = df_cpm.loc[prmtr_id][4]

prmtrs_expr['prmtr']=prmtrs_expr['prmtr'].astype('category')
prmtrs_expr['prmtr_num']=prmtrs_expr['prmtr_num'].astype('category')
prmtrs_expr['geneID']=prmtrs_expr['geneID'].astype('category')
prmtrs_expr['coef']=prmtrs_expr['coef'].astype(float)
prmtrs_expr['cntrb']=prmtrs_expr['cntrb'].astype(float)
prmtrs_expr_df1_uns=prmtrs_expr

In [ ]:
rcParams['figure.figsize'] = 12,12
rcParams["patch.force_edgecolor"] = True
ax = sns.violinplot(data = prmtrs_expr_df1_uns, x = 'prmtr_num', y = "coef", inner = 'quartile', palette = 'Set3', saturation = 1)
fig = ax.get_figure()
fig.patch.set_alpha(0)
fig.savefig("prmtrs_expr_df1_uns_coef.png", bbox_inches='tight', pad_inches =0)

In [ ]:
rcParams['figure.figsize'] = 12,12
rcParams["patch.force_edgecolor"] = True
ax = sns.violinplot(data = prmtrs_expr_df1_uns, x = 'prmtr_num', y = "cntrb", inner = 'quartile', palette = 'Set3', saturation = 1)
fig = ax.get_figure()
fig.patch.set_alpha(0)
fig.savefig("prmtrs_expr_df1_uns_cntrb.png", bbox_inches='tight', pad_inches =0)

In [ ]:
df1_result_dict = get_df_dict(df1_result)
prmtrs_1 = []
prmtrs_2 = []
prmtrs_3 = []
prmtrs_4 = []
prmtrs_5 = []
for key in df1_result_dict:
    prmtrs_list = df1_result_dict.get(key)
    prmtrs_list.sort()
    item = 0
    for i in prmtrs_list:
        item += 1
        if item == 1:
            prmtrs_1.append(i)
        if item == 2:
            prmtrs_2.append(i)
        if item == 3:
            prmtrs_3.append(i)
        if item == 4:
            prmtrs_4.append(i)
        if item >= 5:
            prmtrs_5.append(i)
prmtrs_1_expr = pd.DataFrame(data={'prmtr': prmtrs_1})
prmtrs_2_expr = pd.DataFrame(data={'prmtr': prmtrs_2})
prmtrs_3_expr = pd.DataFrame(data={'prmtr': prmtrs_3})
prmtrs_4_expr = pd.DataFrame(data={'prmtr': prmtrs_4})
prmtrs_5_expr = pd.DataFrame(data={'prmtr': prmtrs_5})

prmtrs_1_expr['prmtr_num']='1st_prmtr'
prmtrs_2_expr['prmtr_num']='2nd_prmtr'
prmtrs_3_expr['prmtr_num']='3rd_prmtr'
prmtrs_4_expr['prmtr_num']='4th_prmtr'
prmtrs_5_expr['prmtr_num']='5th-7th_prmtr'

prmtrs_expr = pd.concat([prmtrs_1_expr, prmtrs_2_expr, prmtrs_3_expr, 
                         prmtrs_4_expr, prmtrs_5_expr], axis = 0, ignore_index = True)
prmtrs_expr['prmtr_num']=prmtrs_expr['prmtr_num']
prmtrs_expr['geneID']=''
prmtrs_expr['coef']=''
prmtrs_expr['cntrb']=''
prmtrs_expr = prmtrs_expr.set_index(keys = 'prmtr', drop=False)

for index, row in prmtrs_expr.iterrows():
    prmtr_id = prmtrs_expr.loc[index][0]
    prmtrs_expr.loc[index][2] = df1_result.loc[prmtr_id][0]
    prmtrs_expr.loc[index][3] = df1_result.loc[prmtr_id][2]

df_cpm = df1_cpm[['ID', 'mean']].set_index(keys = 'ID', drop=False)
df_cpm = df_cpm[df_cpm.index.isin(prmtrs_expr.prmtr.tolist())]
df_cpm[['gene']] = ''
df_cpm[['sum']] = ''
for index, row in df_cpm.iterrows():
    df_cpm['gene'][index] = prmtrs_expr.loc[index][2]

cpm_dict = {}
for index, row in df_cpm.iterrows():
    gene_id=df_cpm.loc[index][2]
    if gene_id not in cpm_dict:
        cpm_dict[gene_id] = list()
    cpm_dict[gene_id].append(df_cpm.loc[index][1])
    
for key in cpm_dict:
    cpm_dict[key] = sum(cpm_dict[key])
    
for index, row in df_cpm.iterrows():
    gene = df_cpm['gene'][index]
    summ = cpm_dict[gene]
    df_cpm['sum'][index] = summ
    
df_cpm['cntrb'] = df_cpm['mean'] / df_cpm['sum']

for index, row in prmtrs_expr.iterrows():
    prmtr_id = prmtrs_expr.loc[index][0]
    prmtrs_expr.loc[index][4] = df_cpm.loc[prmtr_id][4]

prmtrs_expr['prmtr']=prmtrs_expr['prmtr'].astype('category')
prmtrs_expr['prmtr_num']=prmtrs_expr['prmtr_num'].astype('category')
prmtrs_expr['geneID']=prmtrs_expr['geneID'].astype('category')
prmtrs_expr['coef']=prmtrs_expr['coef'].astype(float)
prmtrs_expr['cntrb']=prmtrs_expr['cntrb'].astype(float)
prmtrs_expr_df1=prmtrs_expr

In [ ]:
rcParams['figure.figsize'] = 12,12
rcParams["patch.force_edgecolor"] = True
ax = sns.violinplot(data = prmtrs_expr_df1, x = 'prmtr_num', y = "coef", inner = 'quartile', palette = 'Set3', saturation = 1)
fig = ax.get_figure()
fig.patch.set_alpha(0)
fig.savefig("prmtrs_expr_df1_coef.png", bbox_inches='tight', pad_inches =0)

In [ ]:
rcParams['figure.figsize'] = 12,12
rcParams["patch.force_edgecolor"] = True
ax = sns.violinplot(data = prmtrs_expr_df1, x = 'prmtr_num', y = "cntrb", inner = 'quartile', palette = 'Set3', saturation = 1)
fig = ax.get_figure()
fig.patch.set_alpha(0)
fig.savefig("prmtrs_expr_df1_cntrb.png", bbox_inches='tight', pad_inches =0)

In [ ]:
df2_result_dict = get_df_dict(df2_result)
prmtrs_1 = []
prmtrs_2 = []
prmtrs_3 = []
prmtrs_4 = []
prmtrs_5 = []
for key in df2_result_dict:
    prmtrs_list = df2_result_dict.get(key)
    prmtrs_list.sort()
    item = 0
    for i in prmtrs_list:
        item += 1
        if item == 1:
            prmtrs_1.append(i)
        if item == 2:
            prmtrs_2.append(i)
        if item == 3:
            prmtrs_3.append(i)
        if item == 4:
            prmtrs_4.append(i)
        if item >= 5:
            prmtrs_5.append(i)
prmtrs_1_expr = pd.DataFrame(data={'prmtr': prmtrs_1})
prmtrs_2_expr = pd.DataFrame(data={'prmtr': prmtrs_2})
prmtrs_3_expr = pd.DataFrame(data={'prmtr': prmtrs_3})
prmtrs_4_expr = pd.DataFrame(data={'prmtr': prmtrs_4})
prmtrs_5_expr = pd.DataFrame(data={'prmtr': prmtrs_5})

prmtrs_1_expr['prmtr_num']='1st_prmtr'
prmtrs_2_expr['prmtr_num']='2nd_prmtr'
prmtrs_3_expr['prmtr_num']='3rd_prmtr'
prmtrs_4_expr['prmtr_num']='4th_prmtr'
prmtrs_5_expr['prmtr_num']='5th-7th_prmtr'

prmtrs_expr = pd.concat([prmtrs_1_expr, prmtrs_2_expr, prmtrs_3_expr, 
                         prmtrs_4_expr, prmtrs_5_expr], axis = 0, ignore_index = True)
prmtrs_expr['prmtr_num']=prmtrs_expr['prmtr_num']
prmtrs_expr['geneID']=''
prmtrs_expr['coef']=''
prmtrs_expr['cntrb']=''
prmtrs_expr = prmtrs_expr.set_index(keys = 'prmtr', drop=False)

for index, row in prmtrs_expr.iterrows():
    prmtr_id = prmtrs_expr.loc[index][0]
    prmtrs_expr.loc[index][2] = df2_result.loc[prmtr_id][0]
    prmtrs_expr.loc[index][3] = df2_result.loc[prmtr_id][2]

df_cpm = df2_cpm[['ID', 'mean']].set_index(keys = 'ID', drop=False)
df_cpm = df_cpm[df_cpm.index.isin(prmtrs_expr.prmtr.tolist())]
df_cpm[['gene']] = ''
df_cpm[['sum']] = ''
for index, row in df_cpm.iterrows():
    df_cpm['gene'][index] = prmtrs_expr.loc[index][2]

cpm_dict = {}
for index, row in df_cpm.iterrows():
    gene_id=df_cpm.loc[index][2]
    if gene_id not in cpm_dict:
        cpm_dict[gene_id] = list()
    cpm_dict[gene_id].append(df_cpm.loc[index][1])
    
for key in cpm_dict:
    cpm_dict[key] = sum(cpm_dict[key])
    
for index, row in df_cpm.iterrows():
    gene = df_cpm['gene'][index]
    summ = cpm_dict[gene]
    df_cpm['sum'][index] = summ
    
df_cpm['cntrb'] = df_cpm['mean'] / df_cpm['sum']

for index, row in prmtrs_expr.iterrows():
    prmtr_id = prmtrs_expr.loc[index][0]
    prmtrs_expr.loc[index][4] = df_cpm.loc[prmtr_id][4]

prmtrs_expr['prmtr']=prmtrs_expr['prmtr'].astype('category')
prmtrs_expr['prmtr_num']=prmtrs_expr['prmtr_num'].astype('category')
prmtrs_expr['geneID']=prmtrs_expr['geneID'].astype('category')
prmtrs_expr['coef']=prmtrs_expr['coef'].astype(float)
prmtrs_expr['cntrb']=prmtrs_expr['cntrb'].astype(float)
prmtrs_expr_df2=prmtrs_expr

In [ ]:
rcParams['figure.figsize'] = 12,12
rcParams["patch.force_edgecolor"] = True
ax = sns.violinplot(data = prmtrs_expr_df2, x = 'prmtr_num', y = "coef", inner = 'quartile', palette = 'Set3', saturation = 1)
fig = ax.get_figure()
fig.patch.set_alpha(0)
fig.savefig("prmtrs_expr_df2_coef.png", bbox_inches='tight', pad_inches =0)

In [ ]:
rcParams['figure.figsize'] = 12,12
rcParams["patch.force_edgecolor"] = True
ax = sns.violinplot(data = prmtrs_expr_df2, x = 'prmtr_num', y = "cntrb", inner = 'quartile', palette = 'Set3', saturation = 1)
fig = ax.get_figure()
fig.patch.set_alpha(0)
fig.savefig("prmtrs_expr_df2_cntrb.png", bbox_inches='tight', pad_inches =0)

In [ ]:
df_result_dict = get_df_dict(df_result)
prmtrs_1 = []
prmtrs_2 = []
prmtrs_3 = []
prmtrs_4 = []
prmtrs_5 = []
for key in df_result_dict:
    prmtrs_list = df_result_dict.get(key)
    prmtrs_list.sort()
    item = 0
    for i in prmtrs_list:
        item += 1
        if item == 1:
            prmtrs_1.append(i)
        if item == 2:
            prmtrs_2.append(i)
        if item == 3:
            prmtrs_3.append(i)
        if item == 4:
            prmtrs_4.append(i)
        if item >= 5:
            prmtrs_5.append(i)
prmtrs_1_expr = pd.DataFrame(data={'prmtr': prmtrs_1})
prmtrs_2_expr = pd.DataFrame(data={'prmtr': prmtrs_2})
prmtrs_3_expr = pd.DataFrame(data={'prmtr': prmtrs_3})
prmtrs_4_expr = pd.DataFrame(data={'prmtr': prmtrs_4})
prmtrs_5_expr = pd.DataFrame(data={'prmtr': prmtrs_5})

prmtrs_1_expr['prmtr_num']='1st_prmtr'
prmtrs_2_expr['prmtr_num']='2nd_prmtr'
prmtrs_3_expr['prmtr_num']='3rd_prmtr'
prmtrs_4_expr['prmtr_num']='4th_prmtr'
prmtrs_5_expr['prmtr_num']='5th-7th_prmtr'

prmtrs_expr = pd.concat([prmtrs_1_expr, prmtrs_2_expr, prmtrs_3_expr, 
                         prmtrs_4_expr, prmtrs_5_expr], axis = 0, ignore_index = True)
prmtrs_expr['prmtr_num']=prmtrs_expr['prmtr_num']
prmtrs_expr['geneID']=''
prmtrs_expr['coef']=''
prmtrs_expr['cntrb']=''
prmtrs_expr = prmtrs_expr.set_index(keys = 'prmtr', drop=False)

for index, row in prmtrs_expr.iterrows():
    prmtr_id = prmtrs_expr.loc[index][0]
    prmtrs_expr.loc[index][2] = df_result.loc[prmtr_id][0]
    prmtrs_expr.loc[index][3] = df_result.loc[prmtr_id][2]

df_cpm = df0_cpm[['ID', 'mean']].set_index(keys = 'ID', drop=False)
df_cpm = df_cpm[df_cpm.index.isin(prmtrs_expr.prmtr.tolist())]
df_cpm[['gene']] = ''
df_cpm[['sum']] = ''
for index, row in df_cpm.iterrows():
    df_cpm['gene'][index] = prmtrs_expr.loc[index][2]

cpm_dict = {}
for index, row in df_cpm.iterrows():
    gene_id=df_cpm.loc[index][2]
    if gene_id not in cpm_dict:
        cpm_dict[gene_id] = list()
    cpm_dict[gene_id].append(df_cpm.loc[index][1])
    
for key in cpm_dict:
    cpm_dict[key] = sum(cpm_dict[key])
    
for index, row in df_cpm.iterrows():
    gene = df_cpm['gene'][index]
    summ = cpm_dict[gene]
    df_cpm['sum'][index] = summ
    
df_cpm['cntrb'] = df_cpm['mean'] / df_cpm['sum']

for index, row in prmtrs_expr.iterrows():
    prmtr_id = prmtrs_expr.loc[index][0]
    prmtrs_expr.loc[index][4] = df_cpm.loc[prmtr_id][4]

prmtrs_expr['prmtr']=prmtrs_expr['prmtr'].astype('category')
prmtrs_expr['prmtr_num']=prmtrs_expr['prmtr_num'].astype('category')
prmtrs_expr['geneID']=prmtrs_expr['geneID'].astype('category')
prmtrs_expr['coef']=prmtrs_expr['coef'].astype(float)
prmtrs_expr['cntrb']=prmtrs_expr['cntrb'].astype(float)
prmtrs_expr_df=prmtrs_expr

In [ ]:
rcParams['figure.figsize'] = 12,12
rcParams["patch.force_edgecolor"] = True
ax = sns.violinplot(data = prmtrs_expr_df, x = 'prmtr_num', y = "coef", inner = 'quartile', palette = 'Set3', saturation = 1)
fig = ax.get_figure()
fig.patch.set_alpha(0)
fig.savefig("prmtrs_expr_df_coef.png", bbox_inches='tight', pad_inches =0)

In [ ]:
rcParams['figure.figsize'] = 12,12
rcParams["patch.force_edgecolor"] = True
ax = sns.violinplot(data = prmtrs_expr_df, x = 'prmtr_num', y = "cntrb", inner = 'quartile', palette = 'Set3', saturation = 1)
fig = ax.get_figure()
fig.patch.set_alpha(0)
fig.savefig("prmtrs_expr_df_cntrb.png", bbox_inches='tight', pad_inches =0)

In [ ]:
df0_result_dict = get_df_dict(df0_result)
prmtrs_1 = []
prmtrs_2 = []
prmtrs_3 = []
prmtrs_4 = []
prmtrs_5 = []
for key in df0_result_dict:
    prmtrs_list = df0_result_dict.get(key)
    prmtrs_list.sort()
    item = 0
    for i in prmtrs_list:
        item += 1
        if item == 1:
            prmtrs_1.append(i)
        if item == 2:
            prmtrs_2.append(i)
        if item == 3:
            prmtrs_3.append(i)
        if item == 4:
            prmtrs_4.append(i)
        if item >= 5:
            prmtrs_5.append(i)
prmtrs_1_expr = pd.DataFrame(data={'prmtr': prmtrs_1})
prmtrs_2_expr = pd.DataFrame(data={'prmtr': prmtrs_2})
prmtrs_3_expr = pd.DataFrame(data={'prmtr': prmtrs_3})
prmtrs_4_expr = pd.DataFrame(data={'prmtr': prmtrs_4})
prmtrs_5_expr = pd.DataFrame(data={'prmtr': prmtrs_5})

prmtrs_1_expr['prmtr_num']='1st_prmtr'
prmtrs_2_expr['prmtr_num']='2nd_prmtr'
prmtrs_3_expr['prmtr_num']='3rd_prmtr'
prmtrs_4_expr['prmtr_num']='4th_prmtr'
prmtrs_5_expr['prmtr_num']='5th-7th_prmtr'

prmtrs_expr = pd.concat([prmtrs_1_expr, prmtrs_2_expr, prmtrs_3_expr, 
                         prmtrs_4_expr, prmtrs_5_expr], axis = 0, ignore_index = True)
prmtrs_expr['prmtr_num']=prmtrs_expr['prmtr_num']
prmtrs_expr['geneID']=''
prmtrs_expr['coef']=''
prmtrs_expr['cntrb']=''
prmtrs_expr = prmtrs_expr.set_index(keys = 'prmtr', drop=False)

for index, row in prmtrs_expr.iterrows():
    prmtr_id = prmtrs_expr.loc[index][0]
    prmtrs_expr.loc[index][2] = df0_result.loc[prmtr_id][0]
    prmtrs_expr.loc[index][3] = df0_result.loc[prmtr_id][2]

df_cpm = df0_cpm[['ID', 'mean']].set_index(keys = 'ID', drop=False)
df_cpm = df_cpm[df_cpm.index.isin(prmtrs_expr.prmtr.tolist())]
df_cpm[['gene']] = ''
df_cpm[['sum']] = ''
for index, row in df_cpm.iterrows():
    df_cpm['gene'][index] = prmtrs_expr.loc[index][2]

cpm_dict = {}
for index, row in df_cpm.iterrows():
    gene_id=df_cpm.loc[index][2]
    if gene_id not in cpm_dict:
        cpm_dict[gene_id] = list()
    cpm_dict[gene_id].append(df_cpm.loc[index][1])
    
for key in cpm_dict:
    cpm_dict[key] = sum(cpm_dict[key])
    
for index, row in df_cpm.iterrows():
    gene = df_cpm['gene'][index]
    summ = cpm_dict[gene]
    df_cpm['sum'][index] = summ
    
df_cpm['cntrb'] = df_cpm['mean'] / df_cpm['sum']

for index, row in prmtrs_expr.iterrows():
    prmtr_id = prmtrs_expr.loc[index][0]
    prmtrs_expr.loc[index][4] = df_cpm.loc[prmtr_id][4]

prmtrs_expr['prmtr']=prmtrs_expr['prmtr'].astype('category')
prmtrs_expr['prmtr_num']=prmtrs_expr['prmtr_num'].astype('category')
prmtrs_expr['geneID']=prmtrs_expr['geneID'].astype('category')
prmtrs_expr['coef']=prmtrs_expr['coef'].astype(float)
prmtrs_expr['cntrb']=prmtrs_expr['cntrb'].astype(float)
prmtrs_expr_df0=prmtrs_expr

In [ ]:
rcParams['figure.figsize'] = 12,12
rcParams["patch.force_edgecolor"] = True
ax = sns.violinplot(data = prmtrs_expr_df0, x = 'prmtr_num', y = "coef", inner = 'quartile', palette = 'Set3', saturation = 1)
fig = ax.get_figure()
fig.patch.set_alpha(0)
fig.savefig("prmtrs_expr_df0_coef.png", bbox_inches='tight', pad_inches =0)

In [ ]:
rcParams['figure.figsize'] = 12,12
rcParams["patch.force_edgecolor"] = True
ax = sns.violinplot(data = prmtrs_expr_df0, x = 'prmtr_num', y = "cntrb", inner = 'quartile', palette = 'Set3', saturation = 1)
fig = ax.get_figure()
fig.patch.set_alpha(0)
fig.savefig("prmtrs_expr_df0_cntrb.png", bbox_inches='tight', pad_inches =0)